In [1]:
import pandas as pd
import numpy as np

In [2]:
df_boundary= pd.read_csv("City_Boundary_layer.csv")
df_boundary.head()

,CITY,the_geom
0,CALGARY,"POLYGON ((-114.013291 51.212425, -114.013277 5..."


In [3]:
df_boundary.columns

Index(['CITY', 'the_geom'], dtype='object')

In [4]:
df_boundary['the_geom']

0    POLYGON ((-114.013291 51.212425, -114.013277 5...
Name: the_geom, dtype: object

In [5]:
df_camera = pd.read_csv("Traffic_Camera_Locations.csv")
df_camera.shape
#df_camera.head()

(126, 5)

In [6]:
df_incidents = pd.read_csv("Traffic_Incidents.csv")
display(df_incidents.head(),df_incidents.shape)

,INCIDENT INFO,DESCRIPTION,START_DT,MODIFIED_DT,QUADRANT,Longitude,Latitude,location,Count,id
0,Westbound McKnight Boulevard approaching John ...,2 vehicle incident.,12/09/2016 04:46:32 PM,12/09/2016 05:16:54 PM,NW,-114.083318,51.097316,"(51.09731625733, -114.083317961464)",1,2016-12-09T16:46:3251.0973162573297-114.083317...
1,20 Avenue at 8 Street NW,2 vehicle incident.,12/09/2016 04:58:23 PM,12/09/2016 05:16:54 PM,NW,-114.081378,51.070539,"(51.070538552637, -114.081377719156)",1,2016-12-09T16:58:2351.0705385526371-114.081377...
2,Sunridge Way at 36 Street NE,There is an incident involving LRT.,12/09/2016 05:14:08 PM,12/09/2016 05:16:54 PM,NE,-113.984934,51.067299,"(51.067298691023, -113.98493374196)",1,2016-12-09T17:14:0851.0672986910231-113.984933...
3,Westbound Stoney approaching Shaganappi Trail NW.,Multi vehicle incident.,12/09/2016 05:16:08 PM,12/09/2016 05:16:53 PM,NW,-114.147933,51.152736,"(51.152736445625, -114.147933369876)",1,2016-12-09T17:16:0851.1527364456253-114.147933...
4,Southbound Nose Hill Drive approaching Crowchi...,Multi vehicle incident.,12/09/2016 05:38:05 PM,12/09/2016 05:55:52 PM,NW,-114.203241,51.119684,"(51.11968378497, -114.203240843777)",1,2016-12-09T17:38:0551.1196837849704-114.203240...


(19788, 10)

In [7]:
df_signals = pd.read_csv("Traffic_Signals.csv")
display(df_signals.head(), df_signals.shape)

,INSTDATE,FIRSTROAD,SECONDROAD,QUADRANT,INT_TYPE,PEDBUTTONS,PED_TIMER,ACCESSIBLE PEDESTRIAN SIGNAL,latitude,longitude,Point,Count
0,1969/04/01,KENSINGTON ROAD,12 STREET,NW,Overhead Flasher,Yes,No,NaN,51.052514,-114.091740,POINT (-114.091739730382 51.052513524679),1
1,2005/02/09,COUNTRY HILLS BOULEVARD,COVENTRY HILLS BOULEVARD,NE,Traffic signal T intersection,Yes,No,NaN,51.154211,-114.052475,POINT (-114.05247454616 51.154210611434),1
2,2000/11/13,McCALL WAY,PEGASUS ROAD,NE,Traffic signal,Yes,No,NaN,51.094081,-114.011478,POINT (-114.011477743314 51.094081325343),1
3,1973/01/01,72 AVENUE,CENTRE STREET,N,Traffic signal,Yes,No,NaN,51.117992,-114.069588,POINT (-114.069587594341 51.11799160169),1
4,1957/01/01,9 AVENUE,15 STREET/17 AVENUE,SE,Traffic signal,Yes,No,NaN,51.037739,-114.025122,POINT (-114.025122275833 51.037738985756),1


(1538, 12)

In [8]:
df_signs = pd.read_csv("Traffic_Signs.csv")
display(df_signs,df_signs.shape)

,BLADE_TYPE,COMMENTS,FACING_CD,FLD_SRC_TXT,INSTDATE,MATERIAL,PL_TY,REUSE,SGN_COUNT_NO,SGN_STA_CD,...,STA_CD,SIZE_CD,SUPPORTTYPE,UNITID,TE_SIGNLOCATION_UNITID,POINT,Ward Boundaries,City Quadrants,Calgary Communities,Ward Boundaries 2013-2017
0,Regulatory,NaN,N,NaN,2007/07/01,NaN,NaN,NaN,1.0,A,...,A,60X75,NaN,1.109184e+09,1109082667,POINT (-114.210957448601 51.0685668243),1.0,1.0,128.0,14.0
1,Parking Restrictions,NaN,W,GPS,2016/08/18,High Intensity,SG_40-06,NaN,0.0,R,...,R,30X30,Signpole,1.110021e+09,1110021182,POINT (-113.935804152861 50.911165903496),4.0,3.0,96.0,8.0
2,Timed Parking,NaN,NE,Field Estimate,NaN,NaN,SG_40-05,NaN,4.0,A,...,R,SPECIAL,Signpole,1.110202e+09,1109526373,POINT (-114.05423120852 51.042027593481),14.0,3.0,260.0,5.0
3,Timed Parking,NaN,NE,Field Estimate,2019/01/16,High Intensity,SG_40-05,NaN,4.0,A,...,A,SPECIAL,Signpole,1.110202e+09,1109286994,POINT (-114.053597489366 51.04135079758),14.0,3.0,260.0,5.0
4,Warning,NaN,W,Field Estimate,2018/09/12,High Intensity,SG_40-57,NaN,0.0,A,...,A,NaN,Signpole,1.110271e+09,1110172249,POINT (-113.935926247512 50.911341384981),4.0,3.0,96.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207515,Street Name,NaN,E,GPS,2005/07/01,High Intensity,NaN,NaN,1.0,A,...,A,30XL,Trunk,1.109144e+09,1109076036,POINT (-114.089566544877 50.91413523827),5.0,1.0,156.0,6.0
207516,Regulatory,NaN,N,Field Estimate,2015/06/10,High Intensity,SG_40-05,NaN,3.0,A,...,A,SPECIAL,Signpole,1.109942e+09,1109941864,POINT (-114.073475226496 51.050465695905),14.0,1.0,262.0,2.0
207517,Parking Restrictions,NaN,SW,Field Estimate,2008/07/03,NaN,SG_40-06,NaN,3.0,A,...,A,30X45,Signpole,1.109276e+09,1109276231,POINT (-114.090109343345 51.052801038311),7.0,2.0,40.0,7.0
207518,Street Name,NaN,NW/SE,GPS,2004/07/01,High Intensity,SG_40-99,NaN,2.0,A,...,A,15XL,Signpole,1.109230e+09,1109092275,POINT (-114.053874443342 50.974157960171),10.0,3.0,174.0,5.0


(207520, 21)

In [75]:
df_volume= pd.read_csv("Traffic_Volumes_for_2018.csv")
display(df_volume, df_volume.shape)
df_volume['multilinestring'][5]

,YEAR,SECNAME,Shape_Leng,VOLUME,multilinestring
0,2018,5AVS12,172.570274,22000,MULTILINESTRING ((-114.06036700906716 51.04831...
1,2018,94AVSEWBON,189.451491,5000,MULTILINESTRING ((-114.06876825342002 50.96863...
2,2018,12STNE29,261.453939,5000,MULTILINESTRING ((-114.03372206187294 51.05323...
3,2018,100AVNE1,804.353006,2000,MULTILINESTRING ((-114.01318801277267 51.14336...
4,2018,106AVSE1,507.320869,14000,MULTILINESTRING ((-113.98128756128924 50.95784...
...,...,...,...,...,...
1764,2018,CNTRYHILLS9DEERFT,715.577099,44000,MULTILINESTRING ((-114.02797586216911 51.15420...
1765,2018,BARLOW13SGLEN,797.555381,27000,MULTILINESTRING ((-113.98198296908507 50.97234...
1766,2018,19STNE1B2,184.577040,20000,MULTILINESTRING ((-114.01338212545333 51.06538...
1767,2018,WALDNGASE,139.327583,7000,MULTILINESTRING ((-114.03567148851693 50.87574...


(1769, 5)

'MULTILINESTRING ((-114.08590674401066 51.052502613990775, -114.08587408872434 51.05215627293381, -114.08586396354688 51.05209382966051, -114.08584829677129 51.05204191652356, -114.08583265584006 51.052004737522566, -114.08581070102306 51.05196538044973))'

In [10]:
df_speed = pd.read_csv("Speed_Limits.csv")
display(df_speed.shape, df_speed.head)

(1452, 6)

<bound method NDFrame.head of      BOUND  CREATED_DT  DISTANCE  SPEED          STREET_NAME  \
0        N         NaN      0.81     60       MILLSIDE BV SE   
1      NaN  2018/10/30       NaN     40  DEERFOOT TR SE RAMP   
2        S  2018/10/23       NaN     60   HARVEST HILLS BV N   
3        N  2019/10/09      4.37     70      CROWCHILD TR SW   
4      NaN  2018/10/30       NaN     80    STONEY TR NE RAMP   
...    ...         ...       ...    ...                  ...   
1447     S  2018/10/29       NaN     80         STONEY TR NW   
1448   N/S  2018/10/31       NaN     70             37 ST SW   
1449     W  2018/11/02       NaN     60     SUN VALLEY BV SE   
1450     E         NaN      1.85     60       JOHN LAURIE BV   
1451   NaN  2018/12/06       NaN     60    STONEY TR SE RAMP   

                                              multiline  
0     MULTILINESTRING ((-114.073657541927 50.9135772...  
1     MULTILINESTRING ((-114.034764352652 50.9962241...  
2     MULTILINESTRING ((-11

In [11]:
df_speed.columns

Index(['BOUND', 'CREATED_DT', 'DISTANCE', 'SPEED', 'STREET_NAME', 'multiline'], dtype='object')

In [12]:
def download_weather_data(station, year, month=1, daily=True):
    """ returns a DataFrame with weather data from climate.weather.gc.ca"""
   # url string with variables station, year and month
    url_template = "https://climate.weather.gc.ca/climate_data/bulk_data_e.html?format=csv&stationID={station}&Year={year}&Month={month}&Day=14&timeframe={time_frame}&submit=Download+Data"

    # replace variables in string with actual values
    if daily:
        url = url_template.format(station = station, year = year, month = month, time_frame = 2)
    else:
        url = url_template.format(station= station, year = year, month = month, time_frame = 1)

    # read data into dataframe, use headers and set Date/Time column as index
    weather_data = pd.read_csv(url, index_col='Date/Time', parse_dates=True)

    # replace the degree symbol in the column names
    weather_data.columns = [col.replace('\xb0', '') for col in weather_data.columns]
    
    return (weather_data)

In [13]:
# getting data 2018
data_df_2018 = download_weather_data(station=50430, year=2018, month=5)


display(data_df_2018.columns, data_df_2018.shape)

Index(['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID', 'Year',
       'Month', 'Day', 'Data Quality', 'Max Temp (C)', 'Max Temp Flag',
       'Min Temp (C)', 'Min Temp Flag', 'Mean Temp (C)', 'Mean Temp Flag',
       'Heat Deg Days (C)', 'Heat Deg Days Flag', 'Cool Deg Days (C)',
       'Cool Deg Days Flag', 'Total Rain (mm)', 'Total Rain Flag',
       'Total Snow (cm)', 'Total Snow Flag', 'Total Precip (mm)',
       'Total Precip Flag', 'Snow on Grnd (cm)', 'Snow on Grnd Flag',
       'Dir of Max Gust (10s deg)', 'Dir of Max Gust Flag',
       'Spd of Max Gust (km/h)', 'Spd of Max Gust Flag'],
      dtype='object')

(365, 30)

# Drawing Map

In [14]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster

In [15]:
# reading csv file containing the coordinates of calgary city
df_boundary = pd.read_csv('City_Boundary_layer.csv')

# reading coordinates
df_coordinates = df_boundary['the_geom']
df_cord = df_coordinates[0][10:].replace(")","")
df_cord_split = df_cord.split(",")

# list of longitudes
lons = [float((x.strip().split())[0]) for x in df_cord_split]

# list of latitudes
lats = [float((x.strip().split())[1]) for x in df_cord_split]

# min and max latitude and longitude values
long_max = max(lons)
long_min = min(lons)
lat_max = max(lats)
lat_min = min(lats)

# coordinates for drawing boundary of calgary
ls = ((lat_max,long_max),(lat_max,long_min),(lat_min,long_min),(lat_min,long_max),(lat_max,long_max))

In [16]:
# central location of calgary city
locat = [51.049999, -114.066666]

# displays the map of Calgary city
m = folium.Map(location = locat, zoom_start=10)

# adding boundary coordinates to the map
folium.PolyLine(ls, color="red", weight=2.5, opacity=1).add_to(m)
m

In [17]:
# dividing into grids
vertical_cord_upper = []
vertical_cord_lower = []
horizontal_cord_left = []
horizontal_cord_right = []

# calculating the coordinates for joining the lines
for i in range(1, 10):
    long = long_min + (i*(long_max - long_min)/10)
    lat = lat_min + (i*(lat_max - lat_min)/10)
    vertical_cord_upper.append((lat_max,long))
    vertical_cord_lower.append((lat_min,long))
    horizontal_cord_left.append((lat, long_min))
    horizontal_cord_right.append((lat, long_max))
    horizontal_cord_right


In [18]:
# plotting the grids
for i in range(0,9):
    vertical_cord = []
    horizontal_cord = []
    vertical_cord.append(vertical_cord_upper[i])
    vertical_cord.append(vertical_cord_lower[i])
    horizontal_cord.append(horizontal_cord_left[i])
    horizontal_cord.append(horizontal_cord_right[i])
    folium.PolyLine(vertical_cord, color="brown", weight=1, opacity=1).add_to(m)
    folium.PolyLine(horizontal_cord, color="brown", weight=1, opacity=1).add_to(m)
m

In [19]:
# get coordinates of the grids
# coordinates are stored as a list of tuples
# Each tuple contains 4 values for lat min, lat max, long min, and long max
grid_cord = []
lat_grid_min = lat_min
long_grid_min = long_min
lat_grid_max = 0
long_grid_max = 0

for i in range (1,11):
    lat_grid_max = lat_grid_min + ((lat_max - lat_min)/10)
    long_grid_min = long_min
    for j in range (1,11):
        long_grid_max = long_grid_min + ((long_max - long_min)/10)
        grid_cord.append((lat_grid_min,lat_grid_max,long_grid_min,long_grid_max))
        long_grid_min = long_grid_max
    lat_grid_min = lat_grid_max
print(grid_cord)
print(len(grid_cord))
        
    
        

[(50.842822, 50.8797823, -114.315796, -114.2702069), (50.842822, 50.8797823, -114.2702069, -114.2246178), (50.842822, 50.8797823, -114.2246178, -114.1790287), (50.842822, 50.8797823, -114.1790287, -114.1334396), (50.842822, 50.8797823, -114.1334396, -114.0878505), (50.842822, 50.8797823, -114.0878505, -114.0422614), (50.842822, 50.8797823, -114.0422614, -113.9966723), (50.842822, 50.8797823, -113.9966723, -113.9510832), (50.842822, 50.8797823, -113.9510832, -113.9054941), (50.842822, 50.8797823, -113.9054941, -113.859905), (50.8797823, 50.916742600000006, -114.315796, -114.2702069), (50.8797823, 50.916742600000006, -114.2702069, -114.2246178), (50.8797823, 50.916742600000006, -114.2246178, -114.1790287), (50.8797823, 50.916742600000006, -114.1790287, -114.1334396), (50.8797823, 50.916742600000006, -114.1334396, -114.0878505), (50.8797823, 50.916742600000006, -114.0878505, -114.0422614), (50.8797823, 50.916742600000006, -114.0422614, -113.9966723), (50.8797823, 50.916742600000006, -113.

In [20]:
# checking the coordinates
feature_group = folium.FeatureGroup("Locations")
for i in range (0,100):
    values = grid_cord[i]
    #print(values)
    lat_test = (values[0] + values[1])/2
    long_test = (values[2] + values[3])/2
    feature_group.add_child(folium.Marker(location=[lat_test,long_test],popup="sayam"))
m.add_child(feature_group)

# Counting number of cameras in each grid

In [69]:
# counting no of cameras in each grid

# dictionary for storing the values
dict_data = {}

# column for adding the number of cameras in each grid
dict_data['num_cameras']= []

# getting the latitudes and longitudes of cameras
df_camera_cords = df_camera[['longitude','latitude']]

# algo for counting the number of cameras in each grid 

# coordinates of each grid
for i in grid_cord:
    count = 0
    for j in range (df_camera.shape[0]):
        
        # checks if camera lies in particular grid
        if (i[0] <= df_camera['latitude'][j] <= i[1]) & (i[2] <= df_camera['longitude'][j] <= i[3]):
            count += 1
    # appending num of cameras for that grid
    dict_data['num_cameras'].append(count)
    
sum(dict_data['num_cameras'])
dict_data

{'num_cameras': [0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  3,
  1,
  2,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  3,
  3,
  2,
  1,
  0,
  0,
  0,
  0,
  0,
  3,
  9,
  5,
  1,
  0,
  0,
  0,
  0,
  0,
  2,
  8,
  22,
  5,
  5,
  0,
  0,
  0,
  0,
  1,
  2,
  3,
  4,
  6,
  6,
  2,
  0,
  0,
  0,
  1,
  2,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  2,
  0,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

# Calculating Average Speed

In [118]:
# finding average speed limit

# reading speed limits
df_speed = pd.read_csv("Speed_Limits.csv")

# df_speed_loc has only speeds and location coordinates
df_speed_loc = df_speed[['SPEED','multiline']]

# removing extra characters from location
df_speed_loc['multiline'] = df_speed_loc['multiline'].apply(lambda x: x[16:].replace("(","").replace(")",""))

# checks for NAN values
df_speed_loc.isnull().any()

# creating lists for latitude, longitude, and speed for 'speed data'
speed_lat_ls =[]
speed_long_ls = []
speed_ls = []

# algo for getting lats, longs, and speed
for i in range (df_speed_loc.shape[0]):
    coordinates = df_speed_loc.loc[i,'multiline'].split(",")
    for j in coordinates:
        cord_space = j.split()
        speed_long_ls.append(cord_space[0])
        speed_lat_ls.append(cord_space[1])
        speed_ls.append(df_speed_loc.loc[i,'SPEED'])
    
print(len(speed_ls))
print(len(speed_long_ls))
print(len(speed_lat_ls))



C:\Users\Hp-user\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,SPEED,multiline
0,60,"-114.073657541927 50.913577283979, -114.073651..."
1,40,"-114.034764352652 50.996224143932, -114.034870..."
2,60,"-114.068566084381 51.166713797578, -114.068594..."
3,70,"-114.117825190175 51.00500229492, -114.1178240..."
4,80,"-113.918850808952 51.062448223076, -113.918657..."
...,...,...
1447,80,"-114.233241861102 51.091535273581, -114.233223..."
1448,70,"-114.140309947853 50.917759033844, -114.140309..."
1449,60,"-114.030210417675 50.904313465895, -114.030854..."
1450,60,"-114.177870798005 51.124175201989, -114.177881..."


# Calculating speeds for each grid

In [134]:
# counting average speed in each grid
pd.set_option('display.max_rows', 500)
# adding speed column
dict_data['speed']= []

# coordinates of each grid
for i in grid_cord:
    speed_count = []
    for j in range (len(speed_lat_ls)):
        # checks if coordinate lies in particular grid, then append that speed
        if (i[0] <= float(speed_lat_ls[j]) <= i[1]) & (i[2] <= float(speed_long_ls[j]) <= i[3]):
             # appending num of cameras for that grid
            speed_count.append(speed_ls[j])
    if len(speed_count)==0:
        avg_speed = 0
    else: 
        avg_speed = sum(speed_count)/len(speed_count)
    dict_data['speed'].append(avg_speed)
   
df = pd.DataFrame(dict_data)
df

,num_cameras,speed
0,0,0.000000
1,0,0.000000
2,0,0.000000
3,0,0.000000
4,0,0.000000
5,1,59.673203
6,1,66.206897
7,0,80.445545
8,0,0.000000
9,0,0.000000
